In [2]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import contextily as ctx

Visualise top 10 primary schools and secondary schools on school zones map

Heat map for suburbs with the highest ranking schools 

In [1]:
final_suburb_counts = gpd.read_file("../data/curated/education_df.csv")
primary_schools_df_final = gpd.read_file("../data/raw/primary_school_locations.csv")
final_secondary_schools_df = gpd.read_file("../data/raw/secondary_school_locations.csv")

NameError: name 'gpd' is not defined

In [ ]:


# Step 1: Load the shapefile
vic_suburbs = gpd.read_file("../data/landing/LOCALITY_POLYGON.shp")

# Step 2: Merge the shapefile with your final dataframe
# Ensure suburb names are matched correctly by converting to lowercase and stripping whitespace
vic_suburbs['suburb'] = vic_suburbs['suburb'].str.lower().str.strip()
final_suburb_counts['suburb'] = final_suburb_counts['suburb'].str.lower().str.strip()

# Merge the shapefile with the education data
vic_education_gdf = vic_suburbs.merge(final_suburb_counts, on='suburb', how='left')

# Step 3: Calculate total education density (counts of institutions)
vic_education_gdf['education_density'] = vic_education_gdf['total_education_count'] / vic_education_gdf.area

# Step 4: Find top 10 primary and secondary schools by rank
top_primary_schools = primary_schools_df_final.nsmallest(10, 'rank')
top_secondary_schools = final_secondary_schools_df.nsmallest(10, 'rank')

# Step 5: Create the heatmap
fig, ax = plt.subplots(1, 1, figsize=(12, 12))

# Plot the education density as a heatmap
vic_education_gdf.plot(column='education_density', cmap='OrRd', linewidth=0.8, ax=ax, edgecolor='0.8', legend=True)

# Plot top primary schools on the map
plt.scatter(top_primary_schools['longitude'], top_primary_schools['latitude'], color='blue', label='Top Primary Schools', s=50)

# Plot top secondary schools on the map
plt.scatter(top_secondary_schools['longitude'], top_secondary_schools['latitude'], color='green', label='Top Secondary Schools', s=50)

# Label top primary schools
for idx, row in top_primary_schools.iterrows():
    ax.text(row['longitude'], row['latitude'], row['school_name'], fontsize=8, color='blue')

# Label top secondary schools
for idx, row in top_secondary_schools.iterrows():
    ax.text(row['longitude'], row['latitude'], row['school_name'], fontsize=8, color='green')

# Set plot title and legend
plt.title('Education Institution Density by Suburb in Victoria')
plt.legend(loc='upper right')

# Display the plot
plt.show()

Create table for tertiary education counts per suburb

Calculate distance from property to nearest primary school / secondary school

In [17]:
sample_properties = pd.read_csv('../data/landing/sample.csv')